In [45]:
# To do analysis on multiple stocks

In [46]:
# libraries 

from getData import get_data, get_ohlcv
from technical_indicator import LR, MACD, RSI, BB, ATR, ADX, RENKO, anomaly, historical_volatility, get_support_resistance_levels, VWAP
from Returns import cummulative_returns, CAGR, volatility, SHARPE, SORTINO, MAXDROWDOWN, CALMAR
from signals import stratgey1_mean_reversion, label_sd_with_signals, label_exit_signals


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import copy
import os

%load_ext autoreload
%autoreload 2
pd.set_option('mode.chained_assignment', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Getting Data

In [47]:
list = pd.read_csv('ind_nifty500list.csv')
symbols = list['Symbol'].to_list()
nifty500 = [sym + '.NS' for sym in symbols]

In [48]:
tickers = nifty500

In [49]:
# Store data in dictionary
ohlcv_dict = {}
for stock in tickers:
    ohlcv_dict[stock] = get_ohlcv(ticker=stock, period = '1y', interval='1d', multi_level_index=False)
    # ohlcv_dict[stock] = ohlcv_dict[stock].between_time('09:35', '16:00')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [50]:
data = copy.deepcopy(ohlcv_dict)

# Calculation on Data

In [51]:
for stock_name in tickers:
    try:
        cummulative_returns(data[stock_name])
        ATR(data[stock_name])
        anomaly(data[stock_name])
        support, resistance = get_support_resistance_levels(data[stock_name])
        data[stock_name]['support'] = support
        data[stock_name]['resistance'] = resistance
        historical_volatility(data[stock_name])
        # historical_volatility called twice? Removed duplicate call
        RSI(data[stock_name])
        VWAP(data[stock_name])
        BB(data[stock_name])
        data[stock_name]['Close'] = round(data[stock_name]['Close'], 2)
        data[stock_name]['Signal'] = data[stock_name].apply(stratgey1_mean_reversion, axis=1)
        data[stock_name]['m_half_50'] = data[stock_name]['Close'].ewm(halflife=50).mean()
        data[stock_name]['Distance_From_Mean'] = data[stock_name]['Close'] - data[stock_name]['m_half_50']
        data[stock_name]=label_sd_with_signals(data[stock_name])
        data[stock_name]=label_exit_signals(data[stock_name])
        print(f'Now processing {stock_name}')
        
    except KeyError:
        print(f"Data for {stock_name} not found, skipping.")
        continue
    except Exception as e:
        print(f"Error processing {stock_name}: {e}, skipping.")
        continue


Now processing 360ONE.NS
Now processing 3MINDIA.NS
Now processing ABB.NS
Now processing ACC.NS
Now processing ACMESOLAR.NS
Now processing AIAENG.NS
Now processing APLAPOLLO.NS
Now processing AUBANK.NS
Now processing AWL.NS
Now processing AADHARHFC.NS
Now processing AARTIIND.NS
Now processing AAVAS.NS
Now processing ABBOTINDIA.NS
Now processing ACE.NS
Now processing ADANIENSOL.NS
Now processing ADANIENT.NS
Now processing ADANIGREEN.NS
Now processing ADANIPORTS.NS
Now processing ADANIPOWER.NS
Now processing ATGL.NS
Now processing ABCAPITAL.NS
Now processing ABFRL.NS
Now processing ABLBL.NS
Now processing ABREL.NS
Now processing ABSLAMC.NS
Now processing AEGISLOG.NS
Now processing AFCONS.NS
Now processing AFFLE.NS
Now processing AJANTPHARM.NS
Now processing AKUMS.NS
Now processing APLLTD.NS
Now processing ALIVUS.NS
Now processing ALKEM.NS
Now processing ALKYLAMINE.NS
Now processing ALOKINDS.NS
Now processing ARE&M.NS
Now processing AMBER.NS
Now processing AMBUJACEM.NS
Now processing ANAND

In [52]:
# data['360ONE.NS']

In [60]:
buy_opportunities = []


for stock_name in tickers:
    try:
        df = data[stock_name]
        df['Date'] = df.index
        filtered = df[df['Label'].isin(['BELOW 1 SD'])].copy()
        # filtered = df[df['Label'].isin(['BELOW 2 SD', 'BELOW 3 SD'])].copy()
        # filtered = df[df['Label'].isin(['BELOW 3 SD'])].copy()
        filtered['Stock'] = stock_name  # Optional: add stock name column
        buy_opportunities.append(filtered)
    except KeyError:
        print(f"Data for {stock_name} not found, skipping.")
        continue
    except Exception as e:
        print(f"Error processing {stock_name}: {e}, skipping.")

buy_opportunity = pd.concat(buy_opportunities, ignore_index=True)


Data for DUMMYRAYMN.NS not found, skipping.


In [63]:
track = buy_opportunity.sort_values(by='Date', ascending=False)
track[track['Date']>'2025-07-10']['Stock'].unique()

array(['DOMS.NS', 'TRIVENI.NS', 'POLYMED.NS', 'UNITDSPR.NS', 'ABFRL.NS',
       'HCLTECH.NS', 'FLUOROCHEM.NS', 'ABLBL.NS', 'SAREGAMA.NS',
       'INDHOTEL.NS', 'FSL.NS', 'SARDAEN.NS', 'PATANJALI.NS', 'PGEL.NS',
       'CHAMBLFERT.NS', 'LUPIN.NS', 'GICRE.NS'], dtype=object)

In [55]:
# buy_opportunity['Stock'].unique()

In [56]:
track[track['Date']>'2025-05-01']['Stock'].unique()


array(['DOMS.NS', 'TRIVENI.NS', 'POLYMED.NS', 'UNITDSPR.NS', 'ABFRL.NS',
       'HCLTECH.NS', 'FLUOROCHEM.NS', 'ABLBL.NS', 'SAREGAMA.NS',
       'INDHOTEL.NS', 'FSL.NS', 'SARDAEN.NS', 'PATANJALI.NS', 'PGEL.NS',
       'CHAMBLFERT.NS', 'LUPIN.NS', 'GICRE.NS', 'VBL.NS', 'UBL.NS',
       'COHANCE.NS', 'MANKIND.NS', 'RAYMOND.NS', 'BLUESTARCO.NS',
       'IPCALAB.NS', 'DIXON.NS', 'BALKRISIND.NS', 'VIJAYA.NS', 'ENRIN.NS',
       'PPLPHARMA.NS', 'KIMS.NS', 'BLS.NS', 'SUNPHARMA.NS', 'GRAVITA.NS',
       'SAGILITY.NS', 'CGCL.NS', 'WIPRO.NS', 'RKFORGE.NS', 'JINDALSAW.NS',
       'NEULANDLAB.NS', 'MUTHOOTFIN.NS', 'ETERNAL.NS', 'CONCORDBIO.NS',
       'WELCORP.NS', 'SYNGENE.NS', 'RAINBOW.NS', 'CENTRALBK.NS',
       'SIEMENS.NS', 'WOCKPHARMA.NS', 'HAPPSTMNDS.NS', 'ANANTRAJ.NS',
       'PRAJIND.NS', 'MINDACORP.NS', 'M&MFIN.NS', 'ANANDRATHI.NS',
       'JUBLPHARMA.NS', 'IOB.NS', 'AJANTPHARM.NS', 'GILLETTE.NS',
       'ECLERX.NS', 'OLECTRA.NS', 'NATIONALUM.NS', 'VEDL.NS',
       'GRANULES.NS', 'CENTUR